In [16]:
import torch
import numpy as np
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [78]:
num_epochs=5
num_classes = 10
batch_size=100
learning_rate= 0.01

In [79]:
train_dataset = torchvision.datasets.MNIST(train=True,
                                           root="../MNIST",
                                          download=True,
                                           transform=transforms.ToTensor()
                                          )
train_dataset[0][0].size()

torch.Size([1, 28, 28])

In [80]:
test_dataset = torchvision.datasets.MNIST(train=False,
                                         root="../MNIST/",
                                         download=True,
                                         transform=transforms.ToTensor())

In [81]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,)

In [82]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [83]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=10):
        super(ConvNet,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,16,kernel_size=5,stride=1,padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
        )
        self.layer2 =nn.Sequential(
            nn.Conv2d(16,32,kernel_size=5,stride=1,padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        # 输入是1个通道的28*28的灰度图，经过两次5*5步长为1的卷积
        self.fc = nn.Linear(7*32*7,num_classes)
    
    def forward(self,x):
        out = self.layer1(x)
#         print(out.size())
        out = self.layer2(out)
#         print(out.size())
        out = out.reshape(out.size(0),-1)
#         print(out.size())
        out = self.fc(out)
        return out

In [84]:
model = ConvNet(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [85]:
length = len(train_loader)
for epoch in range(num_epochs):
    for j ,(images,labels) in enumerate(train_loader):
        images = images.to(device)
        label=labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs,label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(j+1)%100==0:
            print("Epoch[{}/{}],Step[{}/{}],loss:{:.4f}".format(epoch+1,num_epochs,j+1,length,loss.item()))

Epoch[1/5],Step[100/600],loss:0.3682
Epoch[1/5],Step[200/600],loss:0.1478
Epoch[1/5],Step[300/600],loss:0.1171
Epoch[1/5],Step[400/600],loss:0.0554
Epoch[1/5],Step[500/600],loss:0.0219
Epoch[1/5],Step[600/600],loss:0.0911
Epoch[2/5],Step[100/600],loss:0.0551
Epoch[2/5],Step[200/600],loss:0.0241
Epoch[2/5],Step[300/600],loss:0.0227
Epoch[2/5],Step[400/600],loss:0.0505
Epoch[2/5],Step[500/600],loss:0.0160
Epoch[2/5],Step[600/600],loss:0.0539
Epoch[3/5],Step[100/600],loss:0.0294
Epoch[3/5],Step[200/600],loss:0.0020
Epoch[3/5],Step[300/600],loss:0.0945
Epoch[3/5],Step[400/600],loss:0.0092
Epoch[3/5],Step[500/600],loss:0.0682
Epoch[3/5],Step[600/600],loss:0.1539
Epoch[4/5],Step[100/600],loss:0.0133
Epoch[4/5],Step[200/600],loss:0.0015
Epoch[4/5],Step[300/600],loss:0.0017
Epoch[4/5],Step[400/600],loss:0.0288
Epoch[4/5],Step[500/600],loss:0.0601
Epoch[4/5],Step[600/600],loss:0.0350
Epoch[5/5],Step[100/600],loss:0.0443
Epoch[5/5],Step[200/600],loss:0.0157
Epoch[5/5],Step[300/600],loss:0.0528
E

In [86]:
model.eval()

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)

In [88]:
with torch.no_grad():
    correct =0
    total = 0
    for images,labels in test_loader:
        images = images.to(device)
        labels=labels.to(device)
        
        outputs = model(images)
        _,pred = torch.max(outputs,1)
        correct += (pred==labels).sum().item()
        total+=labels.size(0)
    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the 10000 test images: 99.11 %


In [ ]:
# Save the model checkpoint
torch.save(model.state_dict(), 'cnn.ckpt')